
<center> <h1> Data Ingestion and cleaning <br> Pandas I/O and Data Cleaup and Preparation

</h1></center>

In [ ]:
pandas_io_tables= pd.read_html('https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html')
len(pandas_io_tables)

In [ ]:
pandas_io_tables[0]

# read_csv

In [ ]:
import pandas as pd


In [ ]:
%%bash
echo -e  "a, b, c\n1, 3, 4\n11, 44, 55\n4, 5, 23"> sample_file.txxxt

In [ ]:
! ls *.txxxt

In [ ]:
!head sample_file.txxxt

This is a delimiter(comma separated) text file. So we can use pandas.read_csv

In [ ]:
df= pd.read_csv('sample_file.txxxt')
df

#  Earth Lab [Figshare](Figshare.com) repository.
download .csv containing monthly average precipitation for Boulder, CO

In [ ]:
pd.read_csv('https://ndownloader.figshare.com/files/12710618')


read_csv has lot of options around **50** to handle complexity of  noisy csv files.

- CSV file has header missing(use **header = None** or give header names using **names = list of header names** )
- Skip some rows( use **skiprows= list of row number**)
- **na_values** to specify a **[ ] of NA values or a dictionary {column_name:na_value}**

In [ ]:
%%bash
echo -e  "11, 233, 123\n1, 3, 4\n11, 44, 55\n4, 5, 23"> sample_file_no_header.txxxt
cat sample_file_no_header.txxxt

In [ ]:
pd.read_csv('sample_file_no_header.txxxt', header= None)

In [ ]:
pd.read_csv('sample_file_no_header.txxxt', names= list('abc'))

# read_table with regular expressions
example from the book

In [ ]:
%%bash
echo -e '            A         B         C\n
 aaa -0.264438 -1.026059 -0.619500\n
 bbb  0.927272  0.302904 -0.032399\n
 ccc -0.264273 -0.386314 -0.217601\n
 ddd -0.871858 -0.348382  1.100491\n' > sample_file_variable_space.txxxt

In [ ]:
!cat sample_file_variable_space.txxxt

In [ ]:
# What regex we should put in sep
pd.read_table('sample_file_variable_space.txxxt', sep=  )

# or read_csv

In [ ]:
pd.read_csv('sample_file_variable_space.txxxt',sep=)

# Reading Text Files in Pieces

let dowload iris dataset form uci machine learning repository

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [ ]:
!head iris.data

Attribute Information:
   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class: 
      -- Iris Setosa
      -- Iris Versicolour
      -- Iris Virginica


In [ ]:
%%bash 
columns="sepal_length,sepal_width,petal_length,petal_width,class"
echo "$columns"| cat - iris.data > iris.csv

In [ ]:
!head iris.csv

In [ ]:
!wc -l iris.csv

Let's say this file is big and can't fit in memory. We can use **read_csv** with **chunksize** option.

Typically a summarization is performed in chunk to perform overall summarization.

In [ ]:
chunks = pd.read_csv('iris.csv', chunksize=20)
print('type of chunks is {}'.format(type(chunks)))
total_sepal_length = 0
num_examples = 0
flower_by_type = pd.Series([])
for chunk in chunks:
    #??? check the type of chunk
    total_sepal_length += chunk['sepal_length'].sum()
    num_examples+= chunk.shape[0]
    
total_sepal_length/num_examples

let's verify this

In [ ]:
iris_df = pd.read_csv('iris.csv')
iris_df.describe()

# Writing/converting data
use **to_** methods like
- to_csv
- to_pickle
- to_sql
- to_latex
- to_html
- t_excel
.
.
.

@TODO to_latex functionality

#  Handling JSON data
- Use pd.read_json to read json file
- use inbult json (loads, dumps) to handle json string and python object

In [ ]:
iris_str_data = iris_df.to_json()
iris_str_data


use json(part of python standard library) to handle json related functionality

## Deserialize  str data to python object

In [ ]:
import json
# Let's Deserialize iris_str_data
iris = json.loads(iris_str_data)
print(type(iris))
iris.keys()

## Convert python data to JSON

In [ ]:
iris_json_obj = json.dumps(iris)
print(type(iris_json_obj))

# Binary dataformat
- pickle.
- HDF5(hierarchical data format) Format. 
    + Useful for large amount of data in data science/machine learning project. Allows efficient read and write for small portion of data. Relevent portion of data can be pulled into memory very fast(**some kind of queue mechanism**).

There are libraries in python(h5py) to handle hdf5 files but pandas has **HDFStore** interface.

In [ ]:
# may have install tables. 
#pip install --upgrade tables
iris_h5_store = pd.HDFStore('iris_data.h5')

In [ ]:
iris_h5_store['iris_class'] = iris_df['class'] 
iris_h5_store['iris_class']

In [ ]:
iris_h5_store.close()

To allow searching use foramt = table

In [ ]:
iris_df.to_hdf('iris.h5', 'iris_table', format= 'table')

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', columns= ['sepal_length', 'class'])

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', where = ['sepal_length<5.0'])

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', where = ['index < 10'])

In [ ]:
iris_df.to_hdf('iris.h5', 'iris_table', format= 'table', data_columns=['sepal_length', 'class'])

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', where = ['sepal_length<5.0'])

# Data cleaning
## Handling Missing Data

In pandas missing data is NA(not available)..

- For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data.
- The built-in Python None value is also treated as NA


## APIs for handling NA
- **dropna:** Filter data based on axis labels, with varying thresholds for how much missing data to tolerate.
-  **fillna:** Fill NA with a value of some interpolation method	

## dropna

https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data

In [ ]:
# As we talked earlier csv files have lot of options, can you figure out the option
#so that ? is treated as NA values
colic_horse_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data', 
                   sep= '\s+', header= None)
colic_horse_df.sample(10)

In [ ]:
colic_horse_df.dropna?

In [ ]:
colic_horse_df.dropna(axis=0, how = 'any')


**how = all** will drop if all row values or column values are NA

## same as this boolean indexing

In [ ]:
import numpy as np
colic_horse_df[ colic_horse_df.notna().all(axis = 1)]

In [ ]:
# Keep  all the row with  number of na values < 4 ??


# Filling In Missing Data

In [ ]:

np.random.seed(1)
df = pd.DataFrame(np.arange(21).reshape(7, 3))
df.iloc[1:4, 1] = np.nan
df.iloc[-2:, 2] = np.nan
df

In [ ]:
df.fillna({1: 0.5, 2: .5})

In [ ]:
df.fillna(method='ffill', limit=2)

In [ ]:
# fill df with median value of each columns


# Data Transformation

# Dropping duplicates

In [ ]:
df = pd.DataFrame(np.arange(12).reshape(4,3), columns=['col'+str(i) for i in range(1, 4)])
df

In [ ]:
np.random.seed(1)
dup_df =pd.concat([df, df.sample(2)])
dup_df = dup_df.reset_index(drop= True)
dup_df.iloc[4, 0] = 33
dup_df

In [ ]:
dup_df.duplicated(subset=['col1', 'col3'])

In [ ]:
dup_df.drop_duplicates(subset=['col1', 'col3'])

# Discretization and Binning

In [ ]:
# exampel from the book
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]


In [ ]:
cats = pd.cut(ages, bins)
print(cats)
type(cats)

In [ ]:
print(cats.codes)
print(cats.categories)




bin counts

In [ ]:
print(pd.value_counts(cats))

## passing bin names

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

# outlier detection
- Unusual dollar amount
- Unusual pixel value in the image

build conditional logic to get boolean index

In [ ]:
np.random.seed(1)
data = pd.DataFrame(2+ np.random.randn(10, 4))
data

In [ ]:
data.describe()

In [ ]:
data[(np.abs(data) > 3).any(1)]

In [ ]:
data[0].quantile(q= .99)

In [ ]:
data[0][np.abs(data[0]) > data[0].quantile(q= .99)] = data[0].quantile(q= .99)
data

# Computing Indicator/Dummy Variables

converting a categorical variable into a “dummy” or “indicator” matrix

Check the book too

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
!unzip ml-latest-small.zip


In [ ]:
!head ml-latest-small/movies.csv

In [ ]:
movie_df = pd.read_csv('ml-latest-small/movies.csv', index_col = 0)
movie_df.head()

In [ ]:
unique_genre =  pd.unique(movie_df.genres.str.split('|').sum())
unique_genre

In [ ]:
# build columns for different genre indicator

In [ ]:
movie_df.head()